In [206]:
import os
import re
import numpy as np
from pymongo import MongoClient
from sympy.combinatorics import Permutation
from sympy import symarray

In [2]:
def cycle(arr, orig = None):
    if arr == orig:
        return []
    if not orig:
        orig = arr
    new_arr = arr[:]
    new_arr.append(new_arr.pop(0))
    return [arr] + cycle(new_arr, orig = orig)

In [3]:
def is_same_parity(perm0, perm1):
    """Check if 2 permutations are of equal parity.

    Assume that both permutation lists are of equal length
    and have the same elements. No need to check for these
    conditions.
    """
    perm1 = perm1[:] ## copy this list so we don't mutate the original

    transCount = 0
    for loc in range(len(perm0) - 1):                         # Do (len - 1) transpositions
        p0 = perm0[loc]
        p1 = perm1[loc]
        if p0 != p1:
            sloc = perm1[loc:].index(p0) + loc          # Find position in perm1
            perm1[loc], perm1[sloc] = p0, p1          # Swap in perm1
            transCount += 1

    # Even number of transpositions means equal parity
    if transCount % 2 == 0:
        return True
    else:
        return False

In [4]:
def volform_parity(ndivs, invol):
    order = list(range(ndivs))
    for x, y in invol:
        temp = order[x]
        order[x] = order[y]
        order[y] = temp
    perm = Permutation(order)
    cycles = cycle(list(range(ndivs)))
    volform_term_parities = [is_same_parity(cycles[i][1:], [i^perm for i in cycles[i^perm][1:]]) for i in range(ndivs)]
    if all(x for x in volform_term_parities):
        return 1
    elif all(not x for x in volform_term_parities):
        return -1
    else:
        return 0

In [262]:
def format_cws(cws):
    return np.array([[int(y) for y in x.split(',')] for x in cws.strip('{}').split('},{')])

def format_invol(invol, ndivs):
    pairs = set(frozenset(int(y.lstrip('D')) - 1 for y in x.split('->')) for x in elem['INVOL'].strip('{}').split(','))
    perm = Permutation()
    for x in pairs:
        perm *= Permutation(*x)
    for x in set(range(ndivs)) - set.union(pairs):
        perm *= Permutation(x)
    return perm

In [6]:
uri = os.getenv('MONGO_URI')
client = MongoClient(uri)
db = client.ToricCY

In [280]:
orientifolds = list(db.INVOL.find({"H11": 3, "VOLFORMPARITY": 0}))
for x in orientifolds:
    x.update(db.TRIANG.find_one({k: x[k] for k in ["POLYID", "GEOMN", "TRIANGN"]}))
    x.update(db.GEOM.find_one({k: x[k] for k in ["POLYID", "GEOMN"]}))
    x.update(db.POLY.find_one({"POLYID": x["POLYID"]}))

In [292]:
db.POLY.find()

['INVOLENUM',
 'POLY',
 'INVOL',
 'SUBCONES',
 'SWISSCHEESE',
 'INDEXES',
 'TRIANG',
 'GEOM']

In [294]:
[x['INDEX'] for x in db.INDEXES.find({'TIER':'GEOM'})]

['POLYID', 'GEOMN']

In [281]:
def volform_eigenvalue(doc):
    cws = format_cws(doc['RESCWS'])
    ndivs = len(cws)
    invol = format_invol(doc['INVOL'], ndivs)

    xs = symarray('x', ndivs)
    xs_perm = xs[[i^invol for i in range(ndivs)]]

    volform_before = (xs @ cws).prod().as_poly()
    volform_after = invol.signature() * (xs_perm @ cws).prod().as_poly()
    volform_eigenval = (volform_after / volform_before).simplify()
    try:
        return int(volform_eigenval)
    except TypeError:
        return 0

In [282]:
a = list(map(volform_eigenvalue, orientifolds))

In [283]:
a

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]